# PREDICTING THE VALUE OF A USED CAR

Problem and questions

- Can i predict a value of a used car by the data acquired from ebay used cars web page?
- What is my target feature?
- Are all the features relevant for the analysis?
- Are there missing or wrong entries?
- What else can i predict? Can i also predict the time that one car stays on the web page?
- Are there outliers?


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from datetime import datetime 
import warnings




Reading the csv file

In [2]:
df=pd.read_csv('autos.csv', header=0, encoding = 'ISO-8859-1')
df.head() 
# if i don't add encoding data, i receive an erro message: UnicodeDecodeError
# to prevent this error, added encoded data type

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/24/16 11:52,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993,manuell,0,golf,150000.0,0,benzin,volkswagen,NaN,3/24/16 0:00,0.0,70435.0,4/7/16 3:16
1,3/24/16 10:58,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011,manuell,190,NaN,125000.0,5,diesel,audi,ja,3/24/16 0:00,0.0,66954.0,4/7/16 1:46
2,3/14/16 12:52,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004,automatik,163,grand,125000.0,8,diesel,jeep,NaN,3/14/16 0:00,0.0,90480.0,4/5/16 12:47
3,3/17/16 16:54,GOLF_4_1_4__3TRER,privat,Angebot,1500.0,test,kleinwagen,2001,manuell,75,golf,150000.0,6,benzin,volkswagen,nein,3/17/16 0:00,0.0,91074.0,3/17/16 17:40
4,3/31/16 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008,manuell,69,fabia,90000.0,7,diesel,skoda,nein,3/31/16 0:00,0.0,60437.0,4/6/16 10:17


## EDA

## Data Wrangling

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371539 entries, 0 to 371538
Data columns (total 20 columns):
dateCrawled            371539 non-null object
name                   371539 non-null object
seller                 371538 non-null object
offerType              371538 non-null object
price                  371538 non-null float64
abtest                 371538 non-null object
vehicleType            333669 non-null object
yearOfRegistration     371538 non-null object
gearbox                351328 non-null object
powerPS                371538 non-null object
model                  351054 non-null object
kilometer              371538 non-null float64
monthOfRegistration    371538 non-null object
fuelType               338151 non-null object
brand                  371537 non-null object
notRepairedDamage      299477 non-null object
dateCreated            371537 non-null object
nrOfPictures           371537 non-null float64
postalCode             371537 non-null float64
lastSeen   

In [4]:
# names of the columns
df.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
df.describe(include='all')

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
count,371539,371539,371538,371538,3.715380e+05,371538,333669,371538.0,351328,371538.0,351054,371538.000000,371538.0,338151,371537,299477,371537,371537.0,371537.000000,371537
unique,15623,233527,3,3,NaN,3,9,245.0,2,1174.0,252,NaN,27.0,7,40,2,114,NaN,NaN,18705
top,3/5/16 14:25,Ford_Fiesta,privat,Angebot,NaN,test,limousine,2000.0,manuell,0.0,golf,NaN,0.0,benzin,volkswagen,nein,4/3/16 0:00,NaN,NaN,4/7/16 6:45
freq,68,657,371534,371525,NaN,192591,95896,22394.0,274219,37244.0,30070,NaN,34364.0,223863,79640,263189,14451,NaN,NaN,708
mean,NaN,NaN,NaN,NaN,1.729584e+04,NaN,NaN,NaN,NaN,NaN,NaN,125617.954096,NaN,NaN,NaN,NaN,NaN,0.0,50820.666402,NaN
std,NaN,NaN,NaN,NaN,3.587905e+06,NaN,NaN,NaN,NaN,NaN,NaN,40113.065296,NaN,NaN,NaN,NaN,NaN,0.0,25799.080292,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,5000.000000,NaN,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,NaN,NaN,NaN,NaN,125000.000000,NaN,NaN,NaN,NaN,NaN,0.0,30459.000000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,NaN,NaN,NaN,NaN,150000.000000,NaN,NaN,NaN,NaN,NaN,0.0,49610.000000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,NaN,NaN,NaN,NaN,150000.000000,NaN,NaN,NaN,NaN,NaN,0.0,71546.000000,NaN


there are some nonsense data such as yearofRegistrain has a value of 1000 or 9999, also there are some outliers. So Data wrangling is important before applying models on the data.

In [6]:
df.describe(include='object')

,dateCrawled,name,seller,offerType,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,lastSeen
count,371539,371539,371538,371538,371538,333669,371538,351328,371538,351054,371538,338151,371537,299477,371537,371537
unique,15623,233527,3,3,3,9,245,2,1174,252,27,7,40,2,114,18705
top,3/5/16 14:25,Ford_Fiesta,privat,Angebot,test,limousine,2000,manuell,0,golf,0,benzin,volkswagen,nein,4/3/16 0:00,4/7/16 6:45
freq,68,657,371534,371525,192591,95896,22394,274219,37244,30070,34364,223863,79640,263189,14451,708


Steps for data wrangling:
- delete the duplicates
- delete unnecessary features
- clean outliers
- change non-english words to english
- 


In [7]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/24/16 11:52,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993,manuell,0,golf,150000.0,0,benzin,volkswagen,NaN,3/24/16 0:00,0.0,70435.0,4/7/16 3:16
1,3/24/16 10:58,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011,manuell,190,NaN,125000.0,5,diesel,audi,ja,3/24/16 0:00,0.0,66954.0,4/7/16 1:46
2,3/14/16 12:52,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004,automatik,163,grand,125000.0,8,diesel,jeep,NaN,3/14/16 0:00,0.0,90480.0,4/5/16 12:47
3,3/17/16 16:54,GOLF_4_1_4__3TRER,privat,Angebot,1500.0,test,kleinwagen,2001,manuell,75,golf,150000.0,6,benzin,volkswagen,nein,3/17/16 0:00,0.0,91074.0,3/17/16 17:40
4,3/31/16 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008,manuell,69,fabia,90000.0,7,diesel,skoda,nein,3/31/16 0:00,0.0,60437.0,4/6/16 10:17


In [8]:
# defining a function, the output should be a dataframe that consist of unique values, null values, and value counts
colons = df.columns

def stat(dataframe, columns):
    print("Total number of rows : ",len(dataframe))
    res = []
    for col in colons:
        NumberOfNull = dataframe[col].isnull().sum()
        NoOfUnique = dataframe[col].nunique()
        ValueCounts = dataframe[col].value_counts().sort_values(ascending=False).head(10)
        res.append((col,NumberOfNull,NoOfUnique,ValueCounts))

            
            
    metric = pd.DataFrame.from_records(res,columns=["Column_names","Null_Counts","Unique_Counts","Value_Counts"])
    return metric

pd.set_option('max_colwidth', 800)
stat(df,colons)

Total number of rows :  371539


,Column_names,Null_Counts,Unique_Counts,Value_Counts
0,dateCrawled,0,15623,"3/5/16 14:25 68 3/5/16 14:26 62 3/5/16 15:48 58 3/5/16 17:49 58 3/5/16 14:49 55 3/16/16 18:49 55 3/27/16 15:50 55 3/21/16 16:50 55 3/29/16 21:50 55 3/20/16 11:50 55 Name: dateCrawled, dtype: int64"
1,name,0,233527,"Ford_Fiesta 657 BMW_318i 627 Opel_Corsa 622 Volkswagen_Golf_1.4 603 BMW_316i 523 BMW_320i 492 Volkswagen_Polo 475 Renault_Twingo 447 Volkswagen_Golf 428 Volkswagen_Golf_1.6 413 Name: name, dtype: int64"
2,seller,1,3,"privat 371534 gewerblich 3 90 1 Name: seller, dtype: int64"
3,offerType,1,3,"Angebot 371525 Gesuch 12 golf 1 Name: offerType, dtype: int64"
4,price,1,5597,"0.0 10778 500.0 5670 1500.0 5394 1000.0 4649 1200.0 4594 2500.0 4438 600.0 3819 3500.0 3792 800.0 3784 2000.0 3432 Name: price, dtype: int64"
5,abtest,1,3,"test 192591 control 178946 4 1 Name: abtest, dtype: int64"
6,vehicleType,37870,9,"limousine 95896 kleinwagen 80026 kombi 67564 bus 30202 cabrio 22899 coupe 19016 suv 14708 andere 3357 benzin 1 Name: vehicleType, dtype: int64"
7,yearOfRegistration,1,245,"2000 22394 1999 20798 2005 20271 2006 18417 2001 18415 2003 18117 2004 18000 2002 17512 1998 16426 2007 16085 Name: yearOfRegistration, dtype: int64"
8,gearbox,20211,2,"manuell 274219 automatik 77109 Name: gearbox, dtype: int64"
9,powerPS,1,1174,"0 37244 75 21991 60 14548 150 14033 140 12383 101 12112 90 11577 116 10949 170 10019 105 9503 Name: powerPS, dtype: int64"



Seller feature : 3 out of 371528 observations are dealer. So this feature can be dropped.

Offer Type feature : 12 out of 371528 observations are Gesuch. So this feature can be dropped.

Number of Pictures feature has all 0 values. So this feature can also be dropped.

PowerPS Feature : 40820 observations are 0. So this needs to be dealt with.

13 unique numbers for the 'kilometer' (mileage) exist. So we can keep them all since there is no outlier.

7 features are discrete, numbers, whereas

12 features are object (string, datetime....)

Shape of the data is 371528x20

"vehicleType, gearbox, model, fuelType, brand, notRepairedDamage" Features have missing values!!!

In [9]:
#DROP THE UNNECESSARY FEATURES/COLUMNS, DELETE DUPLICATES
df = df.drop(['seller','offerType','nrOfPictures','abtest', 'monthOfRegistration'], axis=1)
df=df.drop_duplicates

In [15]:
df = df.replace(['nein','ja','benzin','andere','elektro','manuell','automatik','kleinwagen','kombi'],
                ['no','yes','gas','other','electric','manual','automatic','smallCar','stationWagon'])

AttributeError: 'function' object has no attribute 'replace'

In [ ]:
# Cars with a registration date newer than 2016 and older than 1970 could be deleted. There are only 1739 old cars and 14680 cars with a wrong new entry.¶
print('Number of Cars with newer entries than 2016 :',(df.yearOfRegistration > 2016).sum())
print('Number of Cars with older entries than 1970 :',(df.yearOfRegistration < 1970).sum())


In [12]:
# OUTLIERS AND NULL/MISSING VALUES¶

years = ((df.yearOfRegistration <= 2016) & (df.yearOfRegistration >= 1970))

price = (df.price <= 100000)

power = ((df.powerPS <= 600) & (df.powerPS >= 5))

dfNoOutlier = df[years & price & power]


df.NoOutlier.isnull().sum()

AttributeError: 'function' object has no attribute 'yearOfRegistration'